In [ ]:
!pip install transformers datasets torch pandas scikit-learn matplotlib seaborn


In [ ]:
# Step 2: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

In [ ]:
# Step 3: Load and explore the dataset
print("📊 Loading dataset...")
dataset = load_dataset("sharjeelyunus/github-issues-dataset")

# Convert to pandas for easier exploration
df = dataset['train'].to_pandas()
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\n📋 First few rows:")
print(df.head())
print(df.tail())

In [ ]:
# Step 4: Data exploration and preprocessing
print("\n🔍 Dataset Info:")
print(df.info())
print("\n📈 Target distribution:")
if 'severity' in df.columns:
    print(df['severity'].value_counts())
elif 'label' in df.columns:
    print(df['label'].value_counts())
else:
    print("Available columns:", df.columns.tolist())
    print("Please check which column contains severity labels")

# Check for missing values
print(f"\n❌ Missing values:\n{df.isnull().sum()}")

In [ ]:
# Step 5: Prepare data for BERT
# Combine title and body for better context (adjust column names as needed)
def prepare_text_data(df):
    """Prepare text data by combining relevant fields"""
    text_columns = []

    # Check available text columns
    if 'title' in df.columns:
        text_columns.append('title')
    if 'body' in df.columns:
        text_columns.append('body')
    if 'description' in df.columns:
        text_columns.append('description')

    if len(text_columns) == 0:
        # If no standard columns, use the first text-like column
        for col in df.columns:
            if df[col].dtype == 'object':
                text_columns.append(col)
                break

    # Combine text fields
    if len(text_columns) > 1:
        df['combined_text'] = df[text_columns].fillna('').apply(
            lambda x: ' '.join(x.astype(str)), axis=1
        )
    else:
        df['combined_text'] = df[text_columns[0]].fillna('')

    return df

df = prepare_text_data(df)

In [ ]:
# Step 6: Define severity mapping (adjust based on your dataset)
def map_severity_labels(df):
    """Map severity labels to numerical values"""
    severity_col = None

    # Find severity column
    for col in ['severity', 'label', 'priority', 'level']:
        if col in df.columns:
            severity_col = col
            break

    if severity_col is None:
        print("⚠️ No severity column found. Creating dummy labels for demo.")
        df['severity'] = np.random.choice(['low', 'medium', 'high'], size=len(df))
        severity_col = 'severity'

    # Create label mapping
    unique_labels = df[severity_col].unique()
    label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
    id_to_label = {idx: label for label, idx in label_to_id.items()}

    df['labels'] = df[severity_col].map(label_to_id)

    print(f"📝 Label mapping: {label_to_id}")
    return df, label_to_id, id_to_label

df, label_to_id, id_to_label = map_severity_labels(df)

In [ ]:
# Step 7: Split the data
print("✂️ Splitting data...")
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['labels']
)

train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42,
    stratify=train_df['labels']
)

print(f"Training set: {len(train_df)} samples")
print(f"Validation set: {len(val_df)} samples")
print(f"Test set: {len(test_df)} samples")

In [ ]:
# Step 8: Initialize BERT tokenizer and model
print("🤖 Loading BERT model...")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_to_id)
)

In [ ]:
# Step 9: Memory-efficient tokenization
def tokenize_data_batch(texts, tokenizer, max_length=256, batch_size=1000):
    """Memory-efficient tokenization with batching"""
    print(f"🔤 Tokenizing {len(texts)} texts in batches of {batch_size}...")

    all_input_ids = []
    all_attention_masks = []

    # Process in smaller batches
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size].tolist()
        print(f"  Processing batch {i//batch_size + 1}/{(len(texts)-1)//batch_size + 1}")

        # Tokenize batch
        batch_encoding = tokenizer(
            batch_texts,
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors="pt"
        )

        # Store results
        all_input_ids.append(batch_encoding['input_ids'])
        all_attention_masks.append(batch_encoding['attention_mask'])

        # Clear memory
        del batch_encoding
        torch.cuda.empty_cache() if torch.cuda.is_available() else None

    # Combine all batches
    final_input_ids = torch.cat(all_input_ids, dim=0)
    final_attention_masks = torch.cat(all_attention_masks, dim=0)

    return {
        'input_ids': final_input_ids,
        'attention_mask': final_attention_masks
    }

# SOLUTION 1: Reduce dataset size for testing
print("🔍 Checking dataset size...")
if len(df) > 20000:
    print(f"⚠️ Large dataset detected ({len(df)} samples). Using subset for training.")
    df_subset = df.sample(n=20000, random_state=42)  # Use 20k samples
    train_df, test_df = train_test_split(df_subset, test_size=0.2, random_state=42, stratify=df_subset['labels'])
    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['labels'])
    print(f"✅ Using subset - Training: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# SOLUTION 2: Clean up memory before tokenization
import gc
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

print("🔤 Starting memory-efficient tokenization...")
# Reduced max_length to save memory (256 instead of 512)
train_encodings = tokenize_data_batch(train_df['combined_text'], tokenizer, max_length=256, batch_size=500)
print("✅ Training data tokenized")

val_encodings = tokenize_data_batch(val_df['combined_text'], tokenizer, max_length=256, batch_size=500)
print("✅ Validation data tokenized")

test_encodings = tokenize_data_batch(test_df['combined_text'], tokenizer, max_length=256, batch_size=500)
print("✅ Test data tokenized")

# Clear memory after tokenization
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

In [ ]:
# Step 10: Create Dataset class
class IssueDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = IssueDataset(train_encodings, train_df['labels'].tolist())
val_dataset = IssueDataset(val_encodings, val_df['labels'].tolist())
test_dataset = IssueDataset(test_encodings, test_df['labels'].tolist())

In [ ]:
# Step 11: Safe training arguments (multiple error fixes)
import os

# Create output directory if it doesn't exist
os.makedirs('./results', exist_ok=True)
os.makedirs('./logs', exist_ok=True)

# SAFE training arguments with error handling
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=2,  # Further reduced to prevent memory issues
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    # Removed potentially problematic arguments
    gradient_accumulation_steps=4,  # Increased to compensate for smaller batch
    remove_unused_columns=False,  # Prevent column removal issues
    report_to=None,  # Disable wandb/tensorboard to avoid errors
)
print("✅ Training arguments created successfully!")


# Verify datasets exist before proceeding
print(f"📊 Dataset sizes:")
print(f"  Training: {len(train_dataset)} samples")
print(f"  Validation: {len(val_dataset)} samples")
print(f"  Test: {len(test_dataset)} samples")

In [ ]:
# Step 12: Define metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions)
    }

In [ ]:
# Step 13: Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Step 14: Train the model
print("🚀 Starting training...")
trainer.train()

In [ ]:

# Step 15: Evaluate on test set
print("📊 Evaluating on test set...")
test_results = trainer.evaluate(test_dataset)
print(f"Test Results: {test_results}")

In [ ]:
# Step 16: Make predictions
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = test_df['labels'].tolist()

In [ ]:
# Step 17: Generate detailed results
print("📈 Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(label_to_id.keys())))

In [ ]:

# Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(label_to_id.keys()),
            yticklabels=list(label_to_id.keys()))
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Step 18: Save the model
print("💾 Saving model...")
model.save_pretrained('./issue-severity-classifier')
tokenizer.save_pretrained('./issue-severity-classifier')

In [ ]:
# Step 19: Create inference pipeline
print("🎯 Creating inference pipeline...")
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)

In [ ]:
# Step 20: Test with sample predictions
def predict_severity(text):
    """Predict severity for a given text"""
    results = classifier(text)
    predictions = {id_to_label[i]: score['score'] for i, score in enumerate(results[0])}
    predicted_label = max(predictions, key=predictions.get)
    confidence = max(predictions.values())

    return predicted_label, confidence, predictions

In [ ]:
# Example predictions
sample_issues = [
    "Critical bug: Application crashes on startup",
    "Enhancement: Add dark mode theme",
    "Minor typo in documentation"
]

print("🔮 Sample Predictions:")
for issue in sample_issues:
    pred_label, confidence, all_scores = predict_severity(issue)
    print(f"\nIssue: '{issue}'")
    print(f"Predicted Severity: {pred_label} (Confidence: {confidence:.3f})")
    print(f"All scores: {all_scores}")

In [ ]:
# Step 21: Model performance summary
print("\n" + "="*50)
print("🎉 MODEL TRAINING COMPLETE!")
print("="*50)
print(f"✅ Final Test Accuracy: {accuracy_score(y_true, y_pred):.3f}")
print(f"📁 Model saved to: './issue-severity-classifier'")
print(f"🏷️ Number of classes: {len(label_to_id)}")
print(f"📊 Training samples: {len(train_df)}")
print(f"🔍 Test samples: {len(test_df)}")

In [ ]:
# Step 22: Test with new, unseen data
new_issues = [
    "	Output file size with -s or -g",
    "The login button is not working on the mobile app.",
    "integrate template special sequences in help output",
    "Download range",
    "The API is returning a 500 error for all requests.",
    "The user interface is slow and unresponsive at times.",
    "The color of the main header is not consistent with the brand guidelines."
]

print("🔮 Predicting severity for new issues:")
for issue in new_issues:
    pred_label, confidence, all_scores = predict_severity(issue)
    print(f"\nIssue: '{issue}'")
    print(f"Predicted Severity: {pred_label} (Confidence: {confidence:.3f})")
    print(f"All scores: {all_scores}")